# Visualize ensemble precipitation outputs

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import imageio
import os, glob
import numpy as np

def boxcox_retransform(data, texp=4):
    # transform prcp to approximate normal distribution
    # mode: box-cox; power-law
    if not isinstance(data, np.ndarray):
        data = np.array(data)
    data[data<-texp] = -texp
    datat = (data / texp + 1) ** texp
    return datat


In [2]:

# Ensemble mean
files = glob.glob('../test_cases/cali2017/outputs/ensembles/LWLRstatic_ensMember_20170201-20170215_*.nc')
files.sort()
ds_ens = xr.open_mfdataset(files, concat_dim='z',combine='nested')
ds_ens = ds_ens.mean(dim='z')
ds_ens = ds_ens.mean(dim='time')

fig_list = [] 

duration = 0.5 # Set the duration of each frame in seconds
loop = 0 # Set the number of loops (0 means infinite)
ensnum = 5

for i in range(ensnum):
    outfigi = f'fig_{i}.png'
    fig_list.append(outfigi)
    
    
    fig, ax = plt.subplots(1, 2, figsize=[9, 4])
    
    # ensemble maen
    axi = ax[0]
    dsi = ds_ens
    dsi.prcp.plot(ax=axi, vmin=0, vmax=50)
    axi.set_xlabel('latitude')
    axi.set_ylabel('longitude')
    axi.set_title(f'Ensemble mean precipitation', fontsize=10)
    
    
    axi = ax[1]
    filei = f'../test_cases/cali2017/outputs/ensembles/LWLRstatic_ensMember_20170201-20170215_{i+1:03}.nc'
    dsi = xr.open_dataset(filei)
    # dsi = dsi.isel(time=5)
    dsi = dsi.mean(dim='time')
    dsi.prcp.plot(ax=axi, vmin=0, vmax=50)
    axi.set_xlabel('latitude')
    axi.set_ylabel('longitude')
    axi.set_title(f'Probabilistic precipitation: Member {i+1}', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(outfigi)
    plt.close()

In [3]:
# generate gif 
save_path = 'california2017_ensemble_prcp.gif'
duration = 0.1
loop = 0
    
with imageio.get_writer(save_path, mode='I', duration=duration, loop=loop) as writer:
    for i in range(len(fig_list)):
        image = imageio.v2.imread(fig_list[i])
        writer.append_data(image)

In [4]:
# remove individual files
for i in range(len(fig_list)):
    os.remove(fig_list[i])